In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import joblib
import time
import itertools
import requests
from bs4 import BeautifulSoup
import time
import os
import pandas as pd
import codecs
from urllib.parse import urljoin
import numpy as np

In [2]:
pref=','.join(map(str,range(1,48)))
pref=pref.split(',')
df = pd.DataFrame(pref,columns=["都道府県"])

In [3]:
#content > div.bg_gray > div:nth-child(1) > div.pageNav > div > div.l_grid_05.text_right > ul > li:nth-child(4) > a
df['ページ数']=''
for i in range(len(df['都道府県'])):
    url="https://www.arbeit-jungle.com/Location{0:02d}/".format(int(df['都道府県'][i]))
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "lxml")
    divs = soup.find_all('a',class_='nav_pager_item')
    list1=list()
    for j in divs:
        href = j.attrs['href'] 
        list1.append(href)
    df['ページ数'][i]=list1

KeyboardInterrupt: 

In [ ]:
dff=df.copy()
for i in range(len(dff['ページ数'])):
    list2=list()
    for j in dff['ページ数'][i]:
        j=re.findall(r'page=([0-9]{1,4})',str(j))
        list2.extend(j)
    dff['ページ数'][i]=list2
for i in range(len(dff['ページ数'])):
    if len(dff['ページ数'][i])==0:
        dff['ページ数'][i]=1
    else:
        dff['ページ数'][i]=dff['ページ数'][i][-1]
dff['ページ数']=dff['ページ数'].astype(int)+1

In [84]:
dff

,都道府県,ページ数
0,1,20
1,2,3
2,3,3
3,4,15
4,5,2
5,6,3
6,7,4
7,8,2
8,9,14
9,10,7


In [97]:
def joblib_url(i):
    url = 0
    new_list = list()
    for z in range(1,dff['ページ数'][i]):
        urls = 'https://www.arbeit-jungle.com/Location{0:02d}/?page={1}'.format(int(dff['都道府県'][i]),z)
        res = requests.get(urls)
        try:
            res.raise_for_status()
            soup = BeautifulSoup(res.text, "lxml")
            divs = soup.find_all('h2',class_='card_jobInfo_title')
            for div in divs:
                try:
                    link = div.find('a')
                    href = link.attrs['href'] 
                    new_list.append(href)
                except:
                    pass
        except:
            pass
    return new_list
            

In [99]:
all_list = list()
resultList = joblib.Parallel(n_jobs=-1, verbose=3)( [joblib.delayed(joblib_url)(i) for i  in range(len(df['都道府県']))])
all_list.extend(resultList) 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   56.8s
[Parallel(n_jobs=-1)]: Done  47 out of  47 | elapsed:  6.1min finished


In [100]:
new_all_list = list(itertools.chain.from_iterable(all_list))
kekka = pd.DataFrame(new_all_list,columns=["url"])
kekka

,url
0,/jobs/Com000092983/Rec002977132/
1,/jobs/Com000000032/Rec002857552/
2,/jobs/Com000000032/Rec002857550/
3,/jobs/Com000000032/Rec002857347/
4,/jobs/Com000049353/Rec002975647/
...,...
10450,/jobs/Com000116327/Rec002972734/
10451,/jobs/Com000116327/Rec002972629/
10452,/jobs/Com000084505/Rec002881037/
10453,/jobs/Com000119988/Rec002778374/


In [101]:
kekka.to_csv('create_baito_url.csv')

In [2]:
kekka=pd.read_csv('create_baito_url.csv')

In [3]:
k2=kekka.drop_duplicates()
k2=k2.reset_index()
url_key=k2['url']

In [66]:
url_key[48]

'/jobs/Com000074977/Rec002974427/'

In [60]:
#oboArea > div.l_container_04 > div:nth-child(2) > div:nth-child(1) > table > tbody > tr:nth-child(7) > td
#oboArea > div.l_container_04 > div:nth-child(2) > div > table > tbody > tr:nth-child(7) > td
#oboArea > div.l_container_04 > div:nth-child(2) > div > table > tbody > tr:nth-child(5) > td
llist = list()
url = "https://www.arbeit-jungle.com/jobs/Com000126074/Rec002977396/"
r = requests.get(url)
dennwa=honnsya=sigoto=kinmuchi=kisaisyokusyu=kyuuyo=""
soup = BeautifulSoup(r.text, "lxml")
kk=soup.select('div[class_="inline"]>p[class_="category_l category-inline text_small"]')
kk
#oboArea > div.l_container_04 > div:nth-child(2) > div > table > tbody > tr:nth-child(7) > th
#oboArea > div.l_container_04 > div:nth-child(2) > div:nth-child(1) > table > tbody > tr:nth-child(7) > th

[]

法人名'	施設名	勤務地住所'	本社住所'	（'本社住所'）'都道府県'	（'本社住所'）'市区町村'	雇用区分'	ページタイトル'	媒体記載職種'	業種'	電話番号'	給与情報'	給与区分'	給与下限'	給与上限'	仕事内容'	取得日'	媒体名'	派遣・紹介企業'	ページurl'

In [52]:
def joblib_get_data(z):
    llist = list()
    url = "https://www.arbeit-jungle.com"+url_key[z]
    r = requests.get(url)
    try:
        r.raise_for_status()
        dennwa=honnsya=sigoto=kinmuchi=kisaisyokusyu=kyuuyo=""
        soup = BeautifulSoup(r.text, "lxml")
        try:
            houzinnmei=soup.select_one("#content > div.bg_greenstripe > div > div:nth-child(1) > p").get_text()
        except:
            houzinnmei=""
        try:
            page=soup.select_one("#content > div.l_content_inner.l_inner_m20 > p").get_text()
        except:
            page=""
        for i in range(10):
            try:
                if soup.select_one("#recruitArea > table > tbody > tr:nth-child({}) > th".format(i)).get_text()=="職種":
                    kisaisyokusyu=soup.select_one("#recruitArea > table > tbody > tr:nth-child({}) > td".format(i)).get_text()
                if soup.select_one("#recruitArea > table > tbody > tr:nth-child({}) > th".format(i)).get_text()=="勤務地":
                    kinmuchi=soup.select_one("#recruitArea > table > tbody > tr:nth-child({}) > td>div".format(i)).get_text()
                if soup.select_one("#recruitArea > table > tbody > tr:nth-child({}) > th".format(i)).get_text()=="給与":
                    kyuuyo=soup.select_one("#recruitArea > table > tbody > tr:nth-child({}) > td".format(i)).get_text()
                if soup.select_one("#recruitArea > table > tbody > tr:nth-child({}) > th".format(i)).get_text()=="仕事内容":
                    sigoto=soup.select_one("#recruitArea > table > tbody > tr:nth-child({}) > td".format(i)).get_text()
            except:
                pass
        for i in range(7):
            try:
                if  soup.select_one("#oboArea > div.bs_inner > table > tbody > tr:nth-child(2) > td > div > table > tbody > tr:nth-child({}) > th".format(i)).get_text()=='所在地':
                    honnsya=soup.select_one('#oboArea > div.bs_inner > table > tbody > tr:nth-child(2) > td > div > table > tbody > tr:nth-child({}) > td'.format(i)).get_text()
        
                if  soup.select_one("#oboArea > div.bs_inner > table > tbody > tr:nth-child(2) > td > div > table > tbody > tr:nth-child({}) > th".format(i)).get_text()=='電話番号':
                    dennwa=soup.select_one('#oboArea > div.bs_inner > table > tbody > tr:nth-child(2) > td > div > table > tbody > tr:nth-child({}) > td > span'.format(i)).get_text()
            except:
                pass
        try:
            koyoukubun=soup.find_all("span",class_="icon_workPR_employment")
        except:
            koyoukubun=''
        if honnsya=='':
            for i in range(10):
                try:
                    if soup.select_one("#oboArea > div.l_container_04 > div:nth-child(2) > div:nth-child(1) > table > tbody > tr:nth-child({}) > th".format(i)).get_text()=='所在地':
                        honnsya=soup.select_one("#oboArea > div.l_container_04 > div:nth-child(2) > div:nth-child(1) > table > tbody > tr:nth-child({}) > td".format(i)).get_text()
                    if soup.select_one("#oboArea > div.l_container_04 > div:nth-child(2) > div > table > tbody > tr:nth-child({}) > th".format(i)).get_text()=='所在地':
                        honnsya=soup.select_one("#oboArea > div.l_container_04 > div:nth-child(2) > div > table > tbody > tr:nth-child({}) > td".format(i)).get_text()
                except:
                    pass
        llist.append(houzinnmei)
        llist.append(kinmuchi)
        llist.append(honnsya)
        llist.append(koyoukubun)
        llist.append(page)
        llist.append(kisaisyokusyu)
        llist.append(dennwa)
        llist.append(kyuuyo)
        llist.append(sigoto)
        llist.append(url)
    except:
        pass
    return llist

In [206]:
all_list = list()
resultList = joblib.Parallel(n_jobs=-1, verbose=3,require='sharedmem',backend="threading")([joblib.delayed(joblib_get_data)(i)  for i in range(len(url_key))])
all_list.extend(resultList)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  5.5min


KeyboardInterrupt: 

In [185]:
df_kekka=pd.DataFrame(all_list,columns=['法人名','勤務地住所','本社住所','雇用区分','ページタイトル','媒体記載職種','電話番号','給与情報','仕事内容','ページurl'])
df_kekka

,法人名,勤務地住所,本社住所,雇用区分,ページタイトル,媒体記載職種,電話番号,給与情報,仕事内容,ページurl
0,株式会社 エーアクション,\n\r\n\t 東京都、大阪府を中心とした全国各地\r\...,東京都港区元赤坂１－１－７ 地図を表示,"[[アルバイト], [パート]]",営業事務募集！！,営業事務,080-6376-5547,\n時給：1200円 ★経験者優遇！未経験者でもできるお仕事です。\n,営業事務【アルバイト】事務・営業サポート・人員配置手配等をしていただくお仕事です。,https://www.arbeit-jungle.com/jobs/Com00009298...
1,株式会社 エフエスユニマネジメント ＜市立函館病院＞,\n\r\n\t 市立函館病院\r\n\t ...,108-0023\r\n 東京都 港区 芝浦3-4...,"[[パート], [契約社員]]",資格が無くても、医療現場で働き、人の役に立てるお仕事です！,手術器材の洗浄・滅菌業務,,\n時給900円～\n,資格や経験が無くても、医療の現場で働き、人の役に立っているなと感じることができるお仕事です。...,https://www.arbeit-jungle.com/jobs/Com00000003...
2,株式会社 エフエスユニマネジメント ＜市立函館病院＞,\n\r\n\t 市立函館病院\r\n\t ...,108-0023\r\n 東京都 港区 芝浦3-4...,"[[パート], [契約社員]]",資格が無くても、医療現場で働き、人の役に立てるお仕事です！,薬品・医療材料の供給や取り揃え、在庫管理,,\n時給900円～\n,資格や経験が無くても、医療の現場で働き、人の役に立っているなと感じることができるお仕事です。...,https://www.arbeit-jungle.com/jobs/Com00000003...
3,株式会社 エフエスユニマネジメント ＜市立函館病院＞,\n\r\n\t 市立函館病院\r\n\t ...,108-0023\r\n 東京都 港区 芝浦3-4...,"[[パート], [契約社員]]",資格が無くても、医療現場で働き、人の役に立てるお仕事です！,リネン業務,,\n時給900円～\n,資格や経験が無くても、医療の現場で働き、人の役に立っているなと感じることができるお仕事です。...,https://www.arbeit-jungle.com/jobs/Com00000003...
4,金太郎花太郎グループ,\n\r\n\t 【群馬エリア】金太郎 群馬太田407号店...,東京都新宿区新宿5-9-24 2F 地図を表示,"[[アルバイト], [パート]]","【緊急大募集！即採用！】☆時給1,100円～ WワークOK！学歴・資格不問！",アルバイトスタッフさん（フロント・清掃）,0120-448-897,"\n【店舗フロント アルバイト】 時給 1,100円～1,550円（昇給随時） [夜間手当有...",フロント（受付）、DVDの返却、お客様の店舗ご案内、客室清掃等,https://www.arbeit-jungle.com/jobs/Com00004935...
...,...,...,...,...,...,...,...,...,...,...
95,株式会社マックスサポート,\n\r\n\t 介護施設ほか\r\n\t ...,北海道札幌市中央区北1条西3丁目3 敷島北一条ビル7Ｆ 地図を表示,"[[アルバイト], [パート]]","介護スタッフ：1月開始のお仕事も多数！入職祝い金5万円！（高時給1,600円〜）",介護スタッフ,,"\n時給1,600円〜\n介護福祉士（時給）:1,600円〜夜勤の場合、日給23,000円〜...",介護施設にて、資格を活かした介護業務全般をお任せします！＜具体的には…＞・入浴、排泄、食事、...,https://www.arbeit-jungle.com/jobs/Com00012786...
96,株式会社マックスサポート,\n\r\n\t 介護施設ほか\r\n\t ...,北海道札幌市中央区北1条西3丁目3 敷島北一条ビル7Ｆ 地図を表示,[[派遣社員]],"看護師：1月開始のお仕事も多数！入職祝い金5万円！（高時給1,800円〜/介護施設での看護師さん）",看護師,,"\n時給1,800円〜1,950円 +交通費別途支給\n正看護師（時給）:1,800円〜1,...",有料老人ホーム、デイサービス、グループホーム、特別養護老人ホーム、サービス付き高齢者向け住宅...,https://www.arbeit-jungle.com/jobs/Com00012786...
97,株式会社マックスサポート,\n\r\n\t 介護施設ほか\r\n\t ...,北海道札幌市中央区北1条西3丁目3 敷島北一条ビル7Ｆ 地図を表示,"[[アルバイト], [パート]]","介護スタッフ：1月開始のお仕事も多数！入職祝い金5万円！（高時給1,600円〜）",介護スタッフ,,"\n時給1,600円〜\n介護福祉士（時給）:1,600円〜夜勤の場合、日給23,000円〜...",介護施設にて、資格を活かした介護業務全般をお任せします！＜具体的には…＞・入浴、排泄、食事、...,https://www.arbeit-jungle.com/jobs/Com00012786...
98,株式会社マックスサポート,\n\r\n\t 介護施設ほか\r\n\t ...,北海道札幌市中央区北1条西3丁目3 敷島北一条ビル7Ｆ 地図を表示,[[派遣社員]],"看護師：1月開始のお仕事も多数！入職祝い金5万円！（高時給1,800円〜/介護施設での看護師さん）",看護師,,"\n時給1,800円〜1,950円 +交通費別途支給\n正看護師（時給）:1,800円〜1,...",有料老人ホーム、デイサービス、グループホーム、特別養護老人ホーム、サービス付き高齢者向け住宅...,https://www.arbeit-jungle.com/jobs/Com00012786...


In [186]:
df_kekka.to_csv("create_baito_kekka.csv")

In [187]:
df_kekka1=pd.read_csv('create_baito_kekka.csv')

In [188]:
df_ka=df_kekka1.copy()

In [189]:

#法人名から（）や＜＞を削除
import jaconv
#法人名をクレンジング：全て全角に変換。法人番号リストの法人名にもあとで同じ処理を加える。
#全角に直す関数を定義
def normalize(s):
    s = str(s)
    s = jaconv.z2h(s,digit=True,ascii=True)
    return s

In [190]:
df_ka['法人名']=df_ka['法人名'].apply(normalize)
df_ka

,Unnamed: 0,法人名,勤務地住所,本社住所,雇用区分,ページタイトル,媒体記載職種,電話番号,給与情報,仕事内容,ページurl
0,0,株式会社 ｴｰｱｸｼｮﾝ,\n\r\n\t 東京都、大阪府を中心とした全国各地\r\...,東京都港区元赤坂１－１－７ 地図を表示,"[<span class=""icon_workPR_employment"">アルバイト</s...",営業事務募集！！,営業事務,080-6376-5547,\n時給：1200円 ★経験者優遇！未経験者でもできるお仕事です。\n,営業事務【アルバイト】事務・営業サポート・人員配置手配等をしていただくお仕事です。,https://www.arbeit-jungle.com/jobs/Com00009298...
1,1,株式会社 ｴﾌｴｽﾕﾆﾏﾈｼﾞﾒﾝﾄ <市立函館病院>,\n\r\n\t 市立函館病院\r\n\t ...,108-0023\r\n 東京都 港区 芝浦3-4...,"[<span class=""icon_workPR_employment"">パート</spa...",資格が無くても、医療現場で働き、人の役に立てるお仕事です！,手術器材の洗浄・滅菌業務,NaN,\n時給900円～\n,資格や経験が無くても、医療の現場で働き、人の役に立っているなと感じることができるお仕事です。...,https://www.arbeit-jungle.com/jobs/Com00000003...
2,2,株式会社 ｴﾌｴｽﾕﾆﾏﾈｼﾞﾒﾝﾄ <市立函館病院>,\n\r\n\t 市立函館病院\r\n\t ...,108-0023\r\n 東京都 港区 芝浦3-4...,"[<span class=""icon_workPR_employment"">パート</spa...",資格が無くても、医療現場で働き、人の役に立てるお仕事です！,薬品・医療材料の供給や取り揃え、在庫管理,NaN,\n時給900円～\n,資格や経験が無くても、医療の現場で働き、人の役に立っているなと感じることができるお仕事です。...,https://www.arbeit-jungle.com/jobs/Com00000003...
3,3,株式会社 ｴﾌｴｽﾕﾆﾏﾈｼﾞﾒﾝﾄ <市立函館病院>,\n\r\n\t 市立函館病院\r\n\t ...,108-0023\r\n 東京都 港区 芝浦3-4...,"[<span class=""icon_workPR_employment"">パート</spa...",資格が無くても、医療現場で働き、人の役に立てるお仕事です！,リネン業務,NaN,\n時給900円～\n,資格や経験が無くても、医療の現場で働き、人の役に立っているなと感じることができるお仕事です。...,https://www.arbeit-jungle.com/jobs/Com00000003...
4,4,金太郎花太郎ｸﾞﾙｰﾌﾟ,\n\r\n\t 【群馬エリア】金太郎 群馬太田407号店...,東京都新宿区新宿5-9-24 2F 地図を表示,"[<span class=""icon_workPR_employment"">アルバイト</s...","【緊急大募集！即採用！】☆時給1,100円～ WワークOK！学歴・資格不問！",アルバイトスタッフさん（フロント・清掃）,0120-448-897,"\n【店舗フロント アルバイト】 時給 1,100円～1,550円（昇給随時） [夜間手当有...",フロント（受付）、DVDの返却、お客様の店舗ご案内、客室清掃等,https://www.arbeit-jungle.com/jobs/Com00004935...
...,...,...,...,...,...,...,...,...,...,...,...
95,95,株式会社ﾏｯｸｽｻﾎﾟｰﾄ,\n\r\n\t 介護施設ほか\r\n\t ...,北海道札幌市中央区北1条西3丁目3 敷島北一条ビル7Ｆ 地図を表示,"[<span class=""icon_workPR_employment"">アルバイト</s...","介護スタッフ：1月開始のお仕事も多数！入職祝い金5万円！（高時給1,600円〜）",介護スタッフ,NaN,"\n時給1,600円〜\n介護福祉士（時給）:1,600円〜夜勤の場合、日給23,000円〜...",介護施設にて、資格を活かした介護業務全般をお任せします！＜具体的には…＞・入浴、排泄、食事、...,https://www.arbeit-jungle.com/jobs/Com00012786...
96,96,株式会社ﾏｯｸｽｻﾎﾟｰﾄ,\n\r\n\t 介護施設ほか\r\n\t ...,北海道札幌市中央区北1条西3丁目3 敷島北一条ビル7Ｆ 地図を表示,"[<span class=""icon_workPR_employment"">派遣社員</sp...","看護師：1月開始のお仕事も多数！入職祝い金5万円！（高時給1,800円〜/介護施設での看護師さん）",看護師,NaN,"\n時給1,800円〜1,950円 +交通費別途支給\n正看護師（時給）:1,800円〜1,...",有料老人ホーム、デイサービス、グループホーム、特別養護老人ホーム、サービス付き高齢者向け住宅...,https://www.arbeit-jungle.com/jobs/Com00012786...
97,97,株式会社ﾏｯｸｽｻﾎﾟｰﾄ,\n\r\n\t 介護施設ほか\r\n\t ...,北海道札幌市中央区北1条西3丁目3 敷島北一条ビル7Ｆ 地図を表示,"[<span class=""icon_workPR_employment"">アルバイト</s...","介護スタッフ：1月開始のお仕事も多数！入職祝い金5万円！（高時給1,600円〜）",介護スタッフ,NaN,"\n時給1,600円〜\n介護福祉士（時給）:1,600円〜夜勤の場合、日給23,000円〜...",介護施設にて、資格を活かした介護業務全般をお任せします！＜具体的には…＞・入浴、排泄、食事、...,https://www.arbeit-jungle.com/jobs/Com00012786...
98,98,株式会社ﾏｯｸｽｻﾎﾟｰﾄ,\n\r\n\t 介護施設ほか\r\n\t ...,北海道札幌市中央区北1条西3丁目3 敷島北一条ビル7Ｆ 地図を表示,"[<span class=""icon_workPR_employment"">派遣社員</sp...","看護師：1月開始のお仕事も多数！入職祝い金5万円！（高時給1,800円〜/介護施設での看護師さん）",看護師,NaN,"\n時給1,800円〜1,950円 +交通費別途支給\n正看護師（時給）:1,800円〜1,...",有料老人ホーム、デイサービス、グループホーム、特別養護老人ホーム、サービス付き高齢者向け住宅...,https://www.arbeit-jungle.com/jobs/Com00012786...


In [191]:
#法人奇麗に
df_ka['法人名']=df_ka['法人名'].str.replace('<','%')
df_ka['法人名']=df_ka['法人名'].str.replace('>','%')
df_ka['法人名']=df_ka['法人名'].str.replace('(','%')
df_ka['法人名']=df_ka['法人名'].str.replace(')','%')
df_ka['法人名']=df_ka['法人名'].str.replace('[','%')
df_ka['法人名']=df_ka['法人名'].str.replace(']','%')


In [192]:
df_ka['法人名']

0                    株式会社 ｴｰｱｸｼｮﾝ
1     株式会社 ｴﾌｴｽﾕﾆﾏﾈｼﾞﾒﾝﾄ %市立函館病院%
2     株式会社 ｴﾌｴｽﾕﾆﾏﾈｼﾞﾒﾝﾄ %市立函館病院%
3     株式会社 ｴﾌｴｽﾕﾆﾏﾈｼﾞﾒﾝﾄ %市立函館病院%
4                    金太郎花太郎ｸﾞﾙｰﾌﾟ
                 ...             
95                  株式会社ﾏｯｸｽｻﾎﾟｰﾄ
96                  株式会社ﾏｯｸｽｻﾎﾟｰﾄ
97                  株式会社ﾏｯｸｽｻﾎﾟｰﾄ
98                  株式会社ﾏｯｸｽｻﾎﾟｰﾄ
99                  株式会社ﾏｯｸｽｻﾎﾟｰﾄ
Name: 法人名, Length: 100, dtype: object

In [193]:
for i in range(len(df_ka['法人名'])):
    k=str(df_ka['法人名'][i])
    j=re.sub('%.+?%','',k)
    df_ka['法人名'][i]=j

C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [194]:
df_ka['法人名']

0            株式会社 ｴｰｱｸｼｮﾝ
1     株式会社 ｴﾌｴｽﾕﾆﾏﾈｼﾞﾒﾝﾄ 
2     株式会社 ｴﾌｴｽﾕﾆﾏﾈｼﾞﾒﾝﾄ 
3     株式会社 ｴﾌｴｽﾕﾆﾏﾈｼﾞﾒﾝﾄ 
4            金太郎花太郎ｸﾞﾙｰﾌﾟ
             ...         
95          株式会社ﾏｯｸｽｻﾎﾟｰﾄ
96          株式会社ﾏｯｸｽｻﾎﾟｰﾄ
97          株式会社ﾏｯｸｽｻﾎﾟｰﾄ
98          株式会社ﾏｯｸｽｻﾎﾟｰﾄ
99          株式会社ﾏｯｸｽｻﾎﾟｰﾄ
Name: 法人名, Length: 100, dtype: object

In [195]:

#都道府県

In [196]:
df_ka['本社住所']

0                                   東京都港区元赤坂１－１－７ 地図を表示
1     108-0023\r\n                      東京都 港区 芝浦3-4...
2     108-0023\r\n                      東京都 港区 芝浦3-4...
3     108-0023\r\n                      東京都 港区 芝浦3-4...
4                               東京都新宿区新宿5-9-24 2F 地図を表示
                            ...                        
95                    北海道札幌市中央区北1条西3丁目3　敷島北一条ビル7Ｆ 地図を表示
96                    北海道札幌市中央区北1条西3丁目3　敷島北一条ビル7Ｆ 地図を表示
97                    北海道札幌市中央区北1条西3丁目3　敷島北一条ビル7Ｆ 地図を表示
98                    北海道札幌市中央区北1条西3丁目3　敷島北一条ビル7Ｆ 地図を表示
99                    北海道札幌市中央区北1条西3丁目3　敷島北一条ビル7Ｆ 地図を表示
Name: 本社住所, Length: 100, dtype: object

In [197]:
for i in range(len(df_ka['本社住所'])):
    df_ka['本社住所'][i]=re.sub('\n','',str(df_ka['本社住所'][i]))
    df_ka['本社住所'][i]=re.sub('\r','',str(df_ka['本社住所'][i]))
    df_ka['本社住所'][i]=re.sub(' ','',str(df_ka['本社住所'][i]))
    df_ka['本社住所'][i]=re.sub('　','',str(df_ka['本社住所'][i]))
    df_ka['本社住所'][i]=re.sub('[0-9]{3}-[0-9]{4}','',str(df_ka['本社住所'][i]))

C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing t

In [198]:
df_ka['都道府県']=''
df_ka['市区町村']=''
for i in range(len(df_ka['本社住所'])):
    if df_ka['本社住所'][i]!='nan':
        df_ka['都道府県'][i]=re.search(r"(大阪府)|(京都府)|(北海道)|(.+県)|(東京都)",str(df_ka['本社住所'][i]))
        df_ka['都道府県'][i]=df_ka['都道府県'][i].group()
        df_ka['市区町村'][i]=re.search(r"(\D+?市)|(\D+?区)|(\D+?町)|(\D+?村)",str(df_ka['本社住所'][i]))
        df_ka['市区町村'][i]=df_ka['市区町村'][i].group()
        df_ka['市区町村'][i]=df_ka['市区町村'][i].replace('京都府','')
        df_ka['市区町村'][i]=df_ka['市区町村'][i].replace('大阪府','')
        df_ka['市区町村'][i]=df_ka['市区町村'][i].replace('北海道','')
        df_ka['市区町村'][i]=re.sub(r'.+県','',df_ka['市区町村'][i])
        df_ka['市区町村'][i]=df_ka['市区町村'][i].replace('東京都','')

C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCo

In [199]:
#給与情報

In [200]:
df_ka['給与情報実験']=df_ka['給与情報'].copy()
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('万円','0000円')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('千円','000円')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('万','')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('(','あ')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace(')','あ')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace(',','')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('：','')
df_ka['給与情報実験']=df_ka['給与情報実験'].str.replace('.','')
df_ka['給与数字']=''
df_ka['給与区分']=''
df_ka['給与上限']=0
df_ka['給与下限']=float("inf")
df_ka['給なし'] = df_ka['給与情報実験'].str.match('(.*給.+円)', na=False)
df_ka['出来高']=df_ka['給与情報実験'].astype(str).str.extract('(出来高)')
df_ka['歩合']=df_ka['給与情報実験'].astype(str).str.extract('(歩合)')
df_ka['給あり']=df_ka['給与情報実験'].astype(str).str.extract('(.*給.+円)')
for i in range (len(df_ka['出来高'])):
    if df_ka['出来高'][i]=='出来高':
        df_ka['給与区分'][i]='出来高'
        df_ka['給与上限'][i]=df_ka['給与下限'][i]=np.nan
    else:
        if df_ka['給なし'][i]==False :
            n=str(df_ka['給与情報実験'][i])
            j=re.sub('あ.+?あ','', n)
            j=re.sub('[0-9]{1,2}時',"",j)
            j=re.sub('[0-9]{1,3}%|[0-9]{1,3}プラス|プラス[0-9]{1,3}|[0-9]{1,3}UP',"",j)
            j=re.sub('交通.+[0-9]+?円',"",j)
            df_ka['給与情報実験'][i]=j
#    m=re.findall(r'.給([0-9]+)円|～([0-9]+)円|([0-9]+)～円',df_ka['e'][i])
            m=re.findall(r'([0-9]{4,9})',df_ka['給与情報実験'][i])
            df_ka['給与数字'][i]=m
            h=re.findall(r'(.給)',df_ka['給与情報実験'][i])
            df_ka['給与区分'][i]=h
        if len(str(df_ka['給あり'][i]))!=0:
            n=str(df_ka['給あり'][i])
            j=re.sub('あ.+?あ','', n)
            j=re.sub('[0-9]{1,2}時',"",j)
            j=re.sub('[0-9]{1,3}%|[0-9]{1,3}プラス|プラス[0-9]{1,3}|[0-9]{1,3}UP',"",j)
            j=re.sub('交通.+[0-9]+?円',"",j)
            df_ka['給あり'][i]=j
#    m=re.findall(r'.給([0-9]+)円|～([0-9]+)円|([0-9]+)～円',df_ka['e'][i])
            m=re.findall(r'([0-9]{3,9}?)円',df_ka['給あり'][i])
            df_ka['給与数字'][i].extend(m)
            h=re.findall(r'(.給)',df_ka['給あり'][i])
            df_ka['給与区分'][i].extend(h)

for i in range(len(df_ka['給与区分'])):
    if  df_ka['出来高'][i]!='出来高' :
        flat =",".join(map(str, df_ka['給与区分'][i]))
        sp=flat.split(',')
        kore=set(sp)
        kore.discard('支給')
        kore.discard('合給')
        kore.discard('昇給')
        kore.discard('有給')
        kore.discard('、給')
        df_ka['給与区分'][i]=kore
        if df_ka['歩合'][i]=='歩合':
            df_ka['給与区分'][i].add('歩合')
        df_ka['給与区分'][i]=list(df_ka['給与区分'][i])
for i in range(len(df_ka['給与数字'])):
    if  df_ka['出来高'][i]!='出来高':
        for j in df_ka["給与数字"][i]:
            k=int(j)
            if k>df_ka['給与上限'][i]:
                df_ka['給与上限'][i]=k
            if k<df_ka['給与下限'][i]:
                df_ka['給与下限'][i]=k
  
   
for i in range(len(df_ka['歩合'])):
    if df_ka['歩合'][i]=='歩合':
        if df_ka['給与区分'][i][0]=='':
            df_ka['給与上限'][i]=df_ka['給与下限'][i]=np.nan
    else:
        if len(df_ka['給与区分'][i])==0:
            df_ka['給与区分']='日給'

C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\motok\anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A valu

In [201]:
df2=df_ka.drop(['Unnamed: 0','給なし', '出来高', '給あり', '給与情報実験', '給与数字','歩合'],axis=1)

In [202]:
df2

,法人名,勤務地住所,本社住所,雇用区分,ページタイトル,媒体記載職種,電話番号,給与情報,仕事内容,ページurl,都道府県,市区町村,給与区分,給与上限,給与下限
0,株式会社 ｴｰｱｸｼｮﾝ,\n\r\n\t 東京都、大阪府を中心とした全国各地\r\...,東京都港区元赤坂１－１－７地図を表示,"[<span class=""icon_workPR_employment"">アルバイト</s...",営業事務募集！！,営業事務,080-6376-5547,\n時給：1200円 ★経験者優遇！未経験者でもできるお仕事です。\n,営業事務【アルバイト】事務・営業サポート・人員配置手配等をしていただくお仕事です。,https://www.arbeit-jungle.com/jobs/Com00009298...,東京都,港区,[時給],1200.0,1200.0
1,株式会社 ｴﾌｴｽﾕﾆﾏﾈｼﾞﾒﾝﾄ,\n\r\n\t 市立函館病院\r\n\t ...,東京都港区芝浦3-4-1グランパークタワー23階,"[<span class=""icon_workPR_employment"">パート</spa...",資格が無くても、医療現場で働き、人の役に立てるお仕事です！,手術器材の洗浄・滅菌業務,NaN,\n時給900円～\n,資格や経験が無くても、医療の現場で働き、人の役に立っているなと感じることができるお仕事です。...,https://www.arbeit-jungle.com/jobs/Com00000003...,東京都,港区,[時給],900.0,900.0
2,株式会社 ｴﾌｴｽﾕﾆﾏﾈｼﾞﾒﾝﾄ,\n\r\n\t 市立函館病院\r\n\t ...,東京都港区芝浦3-4-1グランパークタワー23階,"[<span class=""icon_workPR_employment"">パート</spa...",資格が無くても、医療現場で働き、人の役に立てるお仕事です！,薬品・医療材料の供給や取り揃え、在庫管理,NaN,\n時給900円～\n,資格や経験が無くても、医療の現場で働き、人の役に立っているなと感じることができるお仕事です。...,https://www.arbeit-jungle.com/jobs/Com00000003...,東京都,港区,[時給],900.0,900.0
3,株式会社 ｴﾌｴｽﾕﾆﾏﾈｼﾞﾒﾝﾄ,\n\r\n\t 市立函館病院\r\n\t ...,東京都港区芝浦3-4-1グランパークタワー23階,"[<span class=""icon_workPR_employment"">パート</spa...",資格が無くても、医療現場で働き、人の役に立てるお仕事です！,リネン業務,NaN,\n時給900円～\n,資格や経験が無くても、医療の現場で働き、人の役に立っているなと感じることができるお仕事です。...,https://www.arbeit-jungle.com/jobs/Com00000003...,東京都,港区,[時給],900.0,900.0
4,金太郎花太郎ｸﾞﾙｰﾌﾟ,\n\r\n\t 【群馬エリア】金太郎 群馬太田407号店...,東京都新宿区新宿5-9-242F地図を表示,"[<span class=""icon_workPR_employment"">アルバイト</s...","【緊急大募集！即採用！】☆時給1,100円～ WワークOK！学歴・資格不問！",アルバイトスタッフさん（フロント・清掃）,0120-448-897,"\n【店舗フロント アルバイト】 時給 1,100円～1,550円（昇給随時） [夜間手当有...",フロント（受付）、DVDの返却、お客様の店舗ご案内、客室清掃等,https://www.arbeit-jungle.com/jobs/Com00004935...,東京都,新宿区,[時給],1550.0,1020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,株式会社ﾏｯｸｽｻﾎﾟｰﾄ,\n\r\n\t 介護施設ほか\r\n\t ...,北海道札幌市中央区北1条西3丁目3敷島北一条ビル7Ｆ地図を表示,"[<span class=""icon_workPR_employment"">アルバイト</s...","介護スタッフ：1月開始のお仕事も多数！入職祝い金5万円！（高時給1,600円〜）",介護スタッフ,NaN,"\n時給1,600円〜\n介護福祉士（時給）:1,600円〜夜勤の場合、日給23,000円〜...",介護施設にて、資格を活かした介護業務全般をお任せします！＜具体的には…＞・入浴、排泄、食事、...,https://www.arbeit-jungle.com/jobs/Com00012786...,北海道,札幌市,"[時給, 日給]",23000.0,1600.0
96,株式会社ﾏｯｸｽｻﾎﾟｰﾄ,\n\r\n\t 介護施設ほか\r\n\t ...,北海道札幌市中央区北1条西3丁目3敷島北一条ビル7Ｆ地図を表示,"[<span class=""icon_workPR_employment"">派遣社員</sp...","看護師：1月開始のお仕事も多数！入職祝い金5万円！（高時給1,800円〜/介護施設での看護師さん）",看護師,NaN,"\n時給1,800円〜1,950円 +交通費別途支給\n正看護師（時給）:1,800円〜1,...",有料老人ホーム、デイサービス、グループホーム、特別養護老人ホーム、サービス付き高齢者向け住宅...,https://www.arbeit-jungle.com/jobs/Com00012786...,北海道,札幌市,[時給],1950.0,1800.0
97,株式会社ﾏｯｸｽｻﾎﾟｰﾄ,\n\r\n\t 介護施設ほか\r\n\t ...,北海道札幌市中央区北1条西3丁目3敷島北一条ビル7Ｆ地図を表示,"[<span class=""icon_workPR_employment"">アルバイト</s...","介護スタッフ：1月開始のお仕事も多数！入職祝い金5万円！（高時給1,600円〜）",介護スタッフ,NaN,"\n時給1,600円〜\n介護福祉士（時給）:1,600円〜夜勤の場合、日給23,000円〜...",介護施設にて、資格を活かした介護業務全般をお任せします！＜具体的には…＞・入浴、排泄、食事、...,https://www.arbeit-jungle.com/jobs/Com00012786...,北海道,札幌市,"[時給, 日給]",23000.0,1600.0
98,株式会社ﾏｯｸｽｻﾎﾟｰﾄ,\n\r\n\t 介護施設ほか\r\n\t ...,北海道札幌市中央区北1条西3丁目3敷島北一条ビル7Ｆ地図を表示,"[<span class=""icon_workPR_employment"">派遣社員</sp...","看護師：1月開始のお仕事も多数！入職祝い金5万円！（高時給1,800円〜/介護施設での看護師さん）",看護師,NaN,"\n時給1,800円〜1,950円 +交通費別途支給\n正看護師（時給）:1,800円〜1,...",有料老人ホーム、デイサービス、グループホーム、特別養護老人ホーム、サービス付き高齢者向け住宅...,https://www.arbeit-jungle.com/jobs/Com00012786...,北海道,札幌市,[時給],1950.0,1800.0


In [203]:
dfrust=df2.reindex(columns=['法人名','施設名','勤務地住所','本社住所','都道府県','市区町村','雇用区分','ページタイトル','媒体記載職種','業種','電話番号','給与情報','給与区分','給与下限','給与上限','仕事内容','取得日','媒体名','派遣・紹介企業','ページurl'])
import datetime
dfrust['取得日']=datetime.date.today( )
dfrust['媒体名']='create_baito'

In [204]:
dfrust

,法人名,施設名,勤務地住所,本社住所,都道府県,市区町村,雇用区分,ページタイトル,媒体記載職種,業種,電話番号,給与情報,給与区分,給与下限,給与上限,仕事内容,取得日,媒体名,派遣・紹介企業,ページurl
0,株式会社 ｴｰｱｸｼｮﾝ,NaN,\n\r\n\t 東京都、大阪府を中心とした全国各地\r\...,東京都港区元赤坂１－１－７地図を表示,東京都,港区,"[<span class=""icon_workPR_employment"">アルバイト</s...",営業事務募集！！,営業事務,NaN,080-6376-5547,\n時給：1200円 ★経験者優遇！未経験者でもできるお仕事です。\n,[時給],1200.0,1200.0,営業事務【アルバイト】事務・営業サポート・人員配置手配等をしていただくお仕事です。,2020-12-17,create_baito,NaN,https://www.arbeit-jungle.com/jobs/Com00009298...
1,株式会社 ｴﾌｴｽﾕﾆﾏﾈｼﾞﾒﾝﾄ,NaN,\n\r\n\t 市立函館病院\r\n\t ...,東京都港区芝浦3-4-1グランパークタワー23階,東京都,港区,"[<span class=""icon_workPR_employment"">パート</spa...",資格が無くても、医療現場で働き、人の役に立てるお仕事です！,手術器材の洗浄・滅菌業務,NaN,NaN,\n時給900円～\n,[時給],900.0,900.0,資格や経験が無くても、医療の現場で働き、人の役に立っているなと感じることができるお仕事です。...,2020-12-17,create_baito,NaN,https://www.arbeit-jungle.com/jobs/Com00000003...
2,株式会社 ｴﾌｴｽﾕﾆﾏﾈｼﾞﾒﾝﾄ,NaN,\n\r\n\t 市立函館病院\r\n\t ...,東京都港区芝浦3-4-1グランパークタワー23階,東京都,港区,"[<span class=""icon_workPR_employment"">パート</spa...",資格が無くても、医療現場で働き、人の役に立てるお仕事です！,薬品・医療材料の供給や取り揃え、在庫管理,NaN,NaN,\n時給900円～\n,[時給],900.0,900.0,資格や経験が無くても、医療の現場で働き、人の役に立っているなと感じることができるお仕事です。...,2020-12-17,create_baito,NaN,https://www.arbeit-jungle.com/jobs/Com00000003...
3,株式会社 ｴﾌｴｽﾕﾆﾏﾈｼﾞﾒﾝﾄ,NaN,\n\r\n\t 市立函館病院\r\n\t ...,東京都港区芝浦3-4-1グランパークタワー23階,東京都,港区,"[<span class=""icon_workPR_employment"">パート</spa...",資格が無くても、医療現場で働き、人の役に立てるお仕事です！,リネン業務,NaN,NaN,\n時給900円～\n,[時給],900.0,900.0,資格や経験が無くても、医療の現場で働き、人の役に立っているなと感じることができるお仕事です。...,2020-12-17,create_baito,NaN,https://www.arbeit-jungle.com/jobs/Com00000003...
4,金太郎花太郎ｸﾞﾙｰﾌﾟ,NaN,\n\r\n\t 【群馬エリア】金太郎 群馬太田407号店...,東京都新宿区新宿5-9-242F地図を表示,東京都,新宿区,"[<span class=""icon_workPR_employment"">アルバイト</s...","【緊急大募集！即採用！】☆時給1,100円～ WワークOK！学歴・資格不問！",アルバイトスタッフさん（フロント・清掃）,NaN,0120-448-897,"\n【店舗フロント アルバイト】 時給 1,100円～1,550円（昇給随時） [夜間手当有...",[時給],1020.0,1550.0,フロント（受付）、DVDの返却、お客様の店舗ご案内、客室清掃等,2020-12-17,create_baito,NaN,https://www.arbeit-jungle.com/jobs/Com00004935...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,株式会社ﾏｯｸｽｻﾎﾟｰﾄ,NaN,\n\r\n\t 介護施設ほか\r\n\t ...,北海道札幌市中央区北1条西3丁目3敷島北一条ビル7Ｆ地図を表示,北海道,札幌市,"[<span class=""icon_workPR_employment"">アルバイト</s...","介護スタッフ：1月開始のお仕事も多数！入職祝い金5万円！（高時給1,600円〜）",介護スタッフ,NaN,NaN,"\n時給1,600円〜\n介護福祉士（時給）:1,600円〜夜勤の場合、日給23,000円〜...","[時給, 日給]",1600.0,23000.0,介護施設にて、資格を活かした介護業務全般をお任せします！＜具体的には…＞・入浴、排泄、食事、...,2020-12-17,create_baito,NaN,https://www.arbeit-jungle.com/jobs/Com00012786...
96,株式会社ﾏｯｸｽｻﾎﾟｰﾄ,NaN,\n\r\n\t 介護施設ほか\r\n\t ...,北海道札幌市中央区北1条西3丁目3敷島北一条ビル7Ｆ地図を表示,北海道,札幌市,"[<span class=""icon_workPR_employment"">派遣社員</sp...","看護師：1月開始のお仕事も多数！入職祝い金5万円！（高時給1,800円〜/介護施設での看護師さん）",看護師,NaN,NaN,"\n時給1,800円〜1,950円 +交通費別途支給\n正看護師（時給）:1,800円〜1,...",[時給],1800.0,1950.0,有料老人ホーム、デイサービス、グループホーム、特別養護老人ホーム、サービス付き高齢者向け住宅...,2020-12-17,create_baito,NaN,https://www.arbeit-jungle.com/jobs/Com00012786...
97,株式会社ﾏｯｸｽｻﾎﾟｰﾄ,NaN,\n\r\n\t 介護施設ほか\r\n\t ...,北海道札幌市中央区北1条西3丁目3敷島北一条ビル7Ｆ地図を表示,北海道,札幌市,"[<span class=""icon_workPR_employment"">アルバイト</s...","介護スタッフ：1月開始のお仕事も多数！入職祝い金5万円！（高時給1,600円〜）",介護スタッフ,NaN,NaN,"\n時給1,600円〜\n介護福祉士（時給）:1,600円〜夜勤の場合、日給23,000円〜...","[時給, 日給]",1600.0,23000.0,介護施設にて、資格を活かした介護業務全般をお任せします！＜具体的には…＞・入浴、排泄、食事、...,2020-12-17,create_baito,NaN,https://www.arbeit-jungle.com/jobs/Com00012786...
98,株式会社ﾏｯｸｽｻﾎﾟｰﾄ,NaN,\n\r\n\t 介護施設ほか\r\n\t ...,北海道札幌市中央区北1条西3丁目3敷島北一条ビル7Ｆ地図を表示,北海道,札幌市,"[<span class=""icon_workPR_employment"">派遣社員</sp...","看護師：1月開始のお仕事も多数！入職祝い金5万円！（高時給1,800円〜/介護施設での看護師さん）",看護師,NaN,NaN,"\n時給1,800円〜1,950円 +交通費別途支給\n正看護師（時給）:1,800円〜1,...",[時給],1800.0,1950.0,有料老人ホーム、デイサービス、グループホーム、特別養護老人ホーム、サービス付き高齢者向け住宅...,2020-12-17,create_baito,NaN,https://www.arbeit-jungle.com/jobs/Com00012786...


In [205]:
dfrust.to_csv('create_baito_{}.csv'.format(datetime.date.today()))